<table border="1" width="99%">
  <tr>
    <td bgcolor="#48a259">
      <h1 style="color: #FFFFFF; text-align: center;">Exploración de datos en Pyspark</h1>
    </td>
  </tr>
</table>

<table width="99%">
  <tr>
    <td bgcolor="#FFBA39">
      <h2 style="color: #000000; text-align: left;">Conversión de .csv a parquet</h2>
    </td>
  </tr>
</table>

In [1]:
from pyspark.sql import SparkSession
# Crear sesión de Spark
spark = SparkSession.builder \
.appName("DataLakeIngestion") \
.getOrCreate()

In [2]:
# Configuración recomendada: tamaño de bloque/fichero ~128 MB
# (esto afecta a cómo Spark divide los datos al escribir)
spark.conf.set("spark.sql.files.maxPartitionBytes", 134217728) # 128 MB
spark.conf.set("spark.sql.parquet.compression.codec", "snappy") # típico en Data Lake# Configuración recomendada: tamaño de bloque/fichero ~128 MB
# (esto afecta a cómo Spark divide los datos al escribir)
spark.conf.set("spark.sql.files.maxPartitionBytes", 134217728) # 128 MB
spark.conf.set("spark.sql.parquet.compression.codec", "snappy") # típico en Data Lake

In [3]:
# Ruta base de tu Data Lake
base_path = "./datalake"
#Ruta base del Origen
base_origen = "./Origen"

#### Lectura de los archivos

In [4]:
import os
# Verificar directorio actual
print("\nDirectorio actual:", os.getcwd())



Directorio actual: /home/jovyan


In [5]:
# Leer CSVs
df_mef_2022 = spark.read.option("header", True).csv(f"{base_origen}/2022-Gasto-Diario.csv")
df_mef_2023 = spark.read.option("header", True).csv(f"{base_origen}/2023-Gasto-Diario.csv")
df_mef_2024 = spark.read.option("header", True).csv(f"{base_origen}/2024-Gasto-Diario.csv")
df_mef_2025 = spark.read.option("header", True).csv(f"{base_origen}/2025-Gasto-Diario.csv")

#### Escritura en formato Data Lake

In [ ]:
# Guardar cada dataset en su carpeta
df_mef_2022.write.mode("overwrite").parquet(f"{base_path}/mef_2022")
df_mef_2023.write.mode("overwrite").parquet(f"{base_path}/mef_2023")
df_mef_2024.write.mode("overwrite").parquet(f"{base_path}/mef_2024")
df_mef_2025.write.mode("overwrite").parquet(f"{base_path}/mef_2025")

#### Abrir parquets creados anteriormente

In [5]:
# Opción 2: Leer todos los archivos Parquet con wildcard (más elegante)
df_mef_todos = spark.read.parquet(f"{base_path}/mef_*")

print("Archivos Parquet leídos con wildcard")
print(f"Total de registros: {df_mef_todos.count()}")

# Mostrar algunas estadísticas
print("\nPrimeras 5 filas:")
df_mef_todos.show(5)

print("\nEsquema del DataFrame:")
df_mef_todos.printSchema()

Archivos Parquet leídos con wildcard
Total de registros: 14275591

Primeras 5 filas:
+-------+-------+--------------+---------------------+------+-------------+------+-------------+--------+---------+--------------------+----------------------+-----------------------------+-------------------+--------------------------+------------------+-------------------------+--------+-------------+--------------------+-------------+--------------------+-----------------+------------------------+---------------------+----------------------------+-------+--------------------+------------------+-------------------------+---------------+----------------------+-----+---------+--------------------+-----------------+------------------------+---------------------+----------------------------+-----+--------------------+------------+--------------------+---------------+----------------------+----------------+--------+--------------------+-----------+--------------------+---------------+---------------------

<table width="99%">
  <tr>
    <td bgcolor="#FFBA39">
      <h2 style="color: #000000; text-align: left;">Análisis Exploratorio de datos</h2>
    </td>
  </tr>
</table>

#### Información Básica del Dataset

In [6]:
# Información básica del DataFrame
print("="*50)
print("📋 INFORMACIÓN GENERAL DEL DATASET")
print("="*50)

# Dimensiones del dataset
print(f"🔢 Número de filas: {df_mef_todos.count():,}")
print(f"🔢 Número de columnas: {len(df_mef_todos.columns)}")

print("\n" + "="*50)
print("📊 ESQUEMA DEL DATASET")
print("="*50)
df_mef_todos.printSchema()

print("\n" + "="*50)
print("👀 PRIMERAS 10 FILAS")
print("="*50)
df_mef_todos.show(10, truncate=False)

📋 INFORMACIÓN GENERAL DEL DATASET
🔢 Número de filas: 14,275,591
🔢 Número de columnas: 63

📊 ESQUEMA DEL DATASET
root
 |-- ANO_EJE: string (nullable = true)
 |-- MES_EJE: string (nullable = true)
 |-- NIVEL_GOBIERNO: string (nullable = true)
 |-- NIVEL_GOBIERNO_NOMBRE: string (nullable = true)
 |-- SECTOR: string (nullable = true)
 |-- SECTOR_NOMBRE: string (nullable = true)
 |-- PLIEGO: string (nullable = true)
 |-- PLIEGO_NOMBRE: string (nullable = true)
 |-- SEC_EJEC: string (nullable = true)
 |-- EJECUTORA: string (nullable = true)
 |-- EJECUTORA_NOMBRE: string (nullable = true)
 |-- DEPARTAMENTO_EJECUTORA: string (nullable = true)
 |-- DEPARTAMENTO_EJECUTORA_NOMBRE: string (nullable = true)
 |-- PROVINCIA_EJECUTORA: string (nullable = true)
 |-- PROVINCIA_EJECUTORA_NOMBRE: string (nullable = true)
 |-- DISTRITO_EJECUTORA: string (nullable = true)
 |-- DISTRITO_EJECUTORA_NOMBRE: string (nullable = true)
 |-- SEC_FUNC: string (nullable = true)
 |-- PROGRAMA_PPTO: string (nullable = t

#### Análisis de Columnas y Tipos de Datos

In [7]:
# Análisis detallado de columnas
print("="*60)
print("📋 ANÁLISIS DETALLADO DE COLUMNAS")
print("="*60)

# Obtener nombres de columnas
columnas = df_mef_todos.columns
print(f"📝 Lista de columnas ({len(columnas)}):")
for i, col in enumerate(columnas, 1):
    print(f"   {i:2d}. {col}")

# Análisis de valores nulos
print(f"\n{'='*60}")
print("🔍 ANÁLISIS DE VALORES NULOS")
print("="*60)

from pyspark.sql.functions import col, sum as spark_sum, when, isnan, isnull

# Contar valores nulos por columna
null_counts = df_mef_todos.select([
    spark_sum(when(col(c).isNull() | isnan(col(c)), 1).otherwise(0)).alias(c) 
    for c in df_mef_todos.columns
]).collect()[0]

total_rows = df_mef_todos.count()

print(f"{'Columna':<30} {'Nulos':<10} {'% Nulos':<10}")
print("-" * 50)
for col_name in df_mef_todos.columns:
    null_count = null_counts[col_name]
    null_percentage = (null_count / total_rows) * 100 if total_rows > 0 else 0
    print(f"{col_name:<30} {null_count:<10} {null_percentage:<10.2f}%")

📋 ANÁLISIS DETALLADO DE COLUMNAS
📝 Lista de columnas (63):
    1. ANO_EJE
    2. MES_EJE
    3. NIVEL_GOBIERNO
    4. NIVEL_GOBIERNO_NOMBRE
    5. SECTOR
    6. SECTOR_NOMBRE
    7. PLIEGO
    8. PLIEGO_NOMBRE
    9. SEC_EJEC
   10. EJECUTORA
   11. EJECUTORA_NOMBRE
   12. DEPARTAMENTO_EJECUTORA
   13. DEPARTAMENTO_EJECUTORA_NOMBRE
   14. PROVINCIA_EJECUTORA
   15. PROVINCIA_EJECUTORA_NOMBRE
   16. DISTRITO_EJECUTORA
   17. DISTRITO_EJECUTORA_NOMBRE
   18. SEC_FUNC
   19. PROGRAMA_PPTO
   20. PROGRAMA_PPTO_NOMBRE
   21. TIPO_ACT_PROY
   22. TIPO_ACT_PROY_NOMBRE
   23. PRODUCTO_PROYECTO
   24. PRODUCTO_PROYECTO_NOMBRE
   25. ACTIVIDAD_ACCION_OBRA
   26. ACTIVIDAD_ACCION_OBRA_NOMBRE
   27. FUNCION
   28. FUNCION_NOMBRE
   29. DIVISION_FUNCIONAL
   30. DIVISION_FUNCIONAL_NOMBRE
   31. GRUPO_FUNCIONAL
   32. GRUPO_FUNCIONAL_NOMBRE
   33. META
   34. FINALIDAD
   35. META_NOMBRE
   36. DEPARTAMENTO_META
   37. DEPARTAMENTO_META_NOMBRE
   38. FUENTE_FINANCIAMIENTO
   39. FUENTE_FINANCIAMIENT

#### Estadísticas Descriptivas

In [8]:
# Estadísticas descriptivas para columnas numéricas
print("="*60)
print("📊 ESTADÍSTICAS DESCRIPTIVAS")
print("="*60)

# Identificar columnas numéricas
numeric_columns = []
for dtype in df_mef_todos.dtypes:
    if dtype[1] in ['int', 'double', 'float', 'bigint', 'decimal']:
        numeric_columns.append(dtype[0])

print(f"🔢 Columnas numéricas detectadas: {len(numeric_columns)}")
print(f"📋 Columnas: {', '.join(numeric_columns)}")

if numeric_columns:
    print(f"\n{'='*80}")
    print("📈 ESTADÍSTICAS PARA COLUMNAS NUMÉRICAS")
    print("="*80)
    
    # Mostrar estadísticas descriptivas
    df_mef_todos.select(numeric_columns).describe().show()
else:
    print("\n⚠️  No se encontraron columnas numéricas en el dataset")

# Análisis de valores únicos para columnas categóricas
print(f"\n{'='*60}")
print("🏷️  ANÁLISIS DE CARDINALIDAD")
print("="*60)

print(f"{'Columna':<30} {'Valores Únicos':<15} {'Tipo':<15}")
print("-" * 60)

for col_name, col_type in df_mef_todos.dtypes:
    unique_count = df_mef_todos.select(col_name).distinct().count()
    print(f"{col_name:<30} {unique_count:<15} {col_type:<15}")

📊 ESTADÍSTICAS DESCRIPTIVAS
🔢 Columnas numéricas detectadas: 0
📋 Columnas: 

⚠️  No se encontraron columnas numéricas en el dataset

🏷️  ANÁLISIS DE CARDINALIDAD
Columna                        Valores Únicos  Tipo           
------------------------------------------------------------
ANO_EJE                        4               string         
MES_EJE                        13              string         
NIVEL_GOBIERNO                 2               string         
NIVEL_GOBIERNO_NOMBRE          3               string         
SECTOR                         4               string         
SECTOR_NOMBRE                  4               string         
PLIEGO                         30              string         
PLIEGO_NOMBRE                  30              string         
SEC_EJEC                       2483            string         
EJECUTORA                      1972            string         
EJECUTORA_NOMBRE               2361            string         
DEPARTAMENTO_EJECUTOR

#### Análisis Temporal

In [9]:
# Análisis temporal del dataset
from pyspark.sql.functions import year, month, dayofmonth, to_date, date_format

print("="*60)
print("📅 ANÁLISIS TEMPORAL")
print("="*60)

# Buscar columnas que podrían ser fechas
date_columns = []
for col_name, col_type in df_mef_todos.dtypes:
    if 'date' in col_name.lower() or 'fecha' in col_name.lower() or col_type in ['date', 'timestamp']:
        date_columns.append(col_name)

print(f"📅 Columnas de fecha detectadas: {date_columns}")

# Si hay columnas de fecha, hacer análisis temporal
if date_columns:
    for date_col in date_columns:
        print(f"\n🔍 Análisis de la columna: {date_col}")
        print("-" * 40)
        
        # Rango de fechas
        date_range = df_mef_todos.select(
            date_col
        ).agg(
            {"*": "min", date_col: "max"}
        ).collect()[0]
        
        print(f"   📊 Fecha mínima: {date_range[0]}")
        print(f"   📊 Fecha máxima: {date_range[1]}")
        
        # Distribución por año
        print(f"\n   📈 Distribución por año:")
        df_mef_todos.withColumn("año", year(col(date_col))) \
                   .groupBy("año") \
                   .count() \
                   .orderBy("año") \
                   .show()

else:
    # Buscar patrones de fecha en columnas de texto
    print("\n🔍 Buscando patrones de fecha en columnas de texto...")
    
    # Mostrar algunas muestras para identificar columnas con fechas
    sample_data = df_mef_todos.limit(5).collect()
    
    for row in sample_data[:2]:
        print(f"\nMuestra de fila:")
        for i, col_name in enumerate(df_mef_todos.columns):
            value = row[i]
            print(f"   {col_name}: {value}")
        break

📅 ANÁLISIS TEMPORAL
📅 Columnas de fecha detectadas: []

🔍 Buscando patrones de fecha en columnas de texto...

Muestra de fila:
   ANO_EJE: 2024
   MES_EJE: 12
   NIVEL_GOBIERNO: M
   NIVEL_GOBIERNO_NOMBRE: GOBIERNOS LOCALES
   SECTOR:  
   SECTOR_NOMBRE:  
   PLIEGO:  
   PLIEGO_NOMBRE:  
   SEC_EJEC: 300270
   EJECUTORA: 030211
   EJECUTORA_NOMBRE: MUNICIPALIDAD DISTRITAL DE POMACOCHA
   DEPARTAMENTO_EJECUTORA: 03
   DEPARTAMENTO_EJECUTORA_NOMBRE: APURIMAC
   PROVINCIA_EJECUTORA: 02
   PROVINCIA_EJECUTORA_NOMBRE: ANDAHUAYLAS
   DISTRITO_EJECUTORA: 11
   DISTRITO_EJECUTORA_NOMBRE: POMACOCHA
   SEC_FUNC: 11
   PROGRAMA_PPTO: 0042
   PROGRAMA_PPTO_NOMBRE: APROVECHAMIENTO DE LOS RECURSOS HIDRICOS PARA USO AGRARIO
   TIPO_ACT_PROY: 3
   TIPO_ACT_PROY_NOMBRE: ACTIVIDAD
   PRODUCTO_PROYECTO: 3000528
   PRODUCTO_PROYECTO_NOMBRE: PRODUCTORES AGRARIOS CON COMPETENCIAS PARA EL APROVECHAMIENTO DEL RECURSO HIDRICO PARA USO AGRARIO
   ACTIVIDAD_ACCION_OBRA: 5004173
   ACTIVIDAD_ACCION_OBRA_NOMBRE: 